Der alte code

In [34]:
import os
from bs4 import BeautifulSoup
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_ollama import OllamaLLM

# === XML-Dateien einlesen ===
xml_files = ["moduldb-pi2.xml", "Modulbook_pi2_de.xml"]
docs = []

for xml_file in xml_files:
    with open(xml_file, "r", encoding="ISO-8859-1") as f:
        xml_content = f.read()

    soup = BeautifulSoup(xml_content, "xml")
    modules = soup.find_all("moduleheader")

    for module in modules:
        title = module.find("title").text.strip() if module.find("title") else "Kein Titel"
        cid = module.find("cid").text.strip() if module.find("cid") else "Kein Kürzel"
        cp = module.find("cp").text.strip() if module.find("cp") else "?"
        convenor = module.find("convenor").text.strip() if module.find("convenor") else "?"

        types = module.find_all("type")
        ctypes = ", ".join(t.text for t in types) if types else "Unbekannt"

        content = f"""
Modul: {title}
Kürzel: {cid}
Leistungspunkte (ECTS): {cp}
Verantwortlich: {convenor}
Veranstaltungstyp(en): {ctypes}
"""
        docs.append(Document(page_content=content.strip()))

# Debug: Beispielinhalt zeigen
if docs:
    print("📄 Beispiel-Inhalt:\n", docs[0].page_content[:500])
else:
    print("⚠️ Keine Module gefunden.")

# === Dokumente splitten ===
splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
chunks = splitter.split_documents(docs)

# === Embeddings & Chroma ===
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma.from_documents(chunks, embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 1})

# === LLM über Ollama (Server) ===
llm = OllamaLLM(
    base_url="http://134.96.217.20:53100",
    model="llama3-70b",
    temperature=0.5,
    top_p=0.8,
    top_k=10,
    repeat_penalty=1.1,
    presence_penalty=1.2
)

# === Prompt ===
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Hier ist ein Auszug aus Moduldaten:\n"
        "{context}\n\n"
        "Beantworte bitte diese Frage auf einfache Weise:\n"
        "{question}\n\n"
        "Antwort:"
    )
)

# === QA-Kette ===
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

# === Chat starten ===
print("🤖 Chatbot gestartet! Tipp 'exit' oder 'quit' zum Beenden.\n")
while True:
    user_input = input("Du: ")
    if user_input.lower() in ["exit", "quit"]:
        print("🤖 Bot: Auf Wiedersehen!")
        break
    try:
        response = qa_chain.run(user_input)
        print("🤖 Bot:", response)
    except Exception as e:
        print(f"⚠️ Fehler: {e}")

📄 Beispiel-Inhalt:
 Modul: Bachelor-Abschlussarbeit
Kürzel: PIB-BT
Leistungspunkte (ECTS): 12
Verantwortlich: Studienleitung
Veranstaltungstyp(en): Unbekannt
🤖 Chatbot gestartet! Tipp 'exit' oder 'quit' zum Beenden.

🤖 Bot: Das Fach "Informatik 1" hat 5 ECTS-Leistungspunkte.
🤖 Bot: Auf Wiedersehen!


Der neue Code

In [45]:
import os
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_ollama import OllamaLLM

# === Inhalte.txt laden ===
docs = []
with open("Inhalte.txt", "r", encoding="utf-8") as f:
    content = f.read()

blocks = content.split("============================================================")
for block in blocks:
    block = block.strip()
    if len(block) > 100:
        docs.append(Document(page_content=block))

print(f"📄 Datei 'Inhalte.txt' geladen. Module erkannt: {len(docs)}")

# === In kleinere Chunks aufteilen (wenn nötig) ===
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = splitter.split_documents(docs)

# === Vektordatenbank mit Embeddings aufbauen ===
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma.from_documents(chunks, embeddings)
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 5})

# === LLM konfigurieren ===
llm = OllamaLLM(
    base_url="http://134.96.217.20:53100",  # ggf. deine Adresse
    model="llama3-70b",  # oder mistral-7b je nach ollama setup
    temperature=0.3
)

# === Prompt definieren ===
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Hier ist ein Auszug aus Modulbeschreibungen:\n"
        "{context}\n\n"
        "Beantworte die folgende Frage **nur auf Basis dieser Informationen**. "
        "Wenn die Antwort nicht im Text steht, sage: 'Nicht enthalten'.\n\n"
        "Frage: {question}\n\n"
        "Antwort:"
    )
)

# === Retrieval-QA-Kette starten ===
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

# === Chat starten ===
print("\n🤖 Chatbot gestartet! Tipp 'exit' oder 'quit' zum Beenden.\n")
while True:
    user_input = input("Du: ")
    if user_input.lower() in ["exit", "quit"]:
        print("🤖 Bot: Auf Wiedersehen!")
        break
    try:
        response = qa_chain.run(user_input)
        print("🤖 Bot:", response)
    except Exception as e:
        print("⚠️ Fehler:", e)


📄 Datei 'Inhalte.txt' geladen. Module erkannt: 3

🤖 Chatbot gestartet! Tipp 'exit' oder 'quit' zum Beenden.

🤖 Bot: Die Fragen können wie folgt beantwortet werden:

1. Was sind die Lernziele im Modul Betriebswirtschaftslehre?
   - Nicht enthalten.

2. Welche Inhalte hat Business Comm. and Intercultural Competence?
   - Die spezifischen Inhalte des Moduls "Business Communication and Intercultural Competence" werden nicht genannt, aber es verwendet Lehrmethoden wie Übungen, Fallstudien und Gamification. Die Literatur bezieht sich jedoch auf Betriebswirtschaftslehre.

3. Was lernt man in Betriebssystemeinführung?
   - Nicht enthalten. Es gibt keine Informationen über ein Modul namens "Betriebssystemeinführung".

4. Wie viele ECTS hat das Modul PIB-BWL?
   - 5 ECTS-Punkte.
🤖 Bot: Auf Wiedersehen!
